In [2]:
import os
import copy
import math
import pybullet as p
import pybullet_data
import numpy as np

In [3]:
class Bike:

    def __init__(self, timeStep=0.01):
        p.connect(p.GUI)
        p.resetSimulation()
        self.urdfRootPath = pybullet_data.getDataPath()
        self.timeStep = timeStep


    def reset(self):
        planeUid = p.loadURDF(os.path.join(self.urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.bike = p.loadURDF(os.path.join(self.urdfRootPath, "bicycle/bike.urdf"),
                           [0, 0, 1],
                           useFixedBase=False)
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.bike)):
            p.setJointMotorControl2(self.bike,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 20.
        self.steeringMultiplier = 0.5
        
    def getActionDimension(self):
        return self.nMotors
    
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.bike)

        observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
    
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        print("targetVelocity")
        print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        print("steeringAngle")
        print(steeringAngle)
        print("maxForce")
        print(self.maxForce)

        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.bike,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.bike,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

In [4]:
r = Bike()
r.reset()
p.setRealTimeSimulation(1)
speed = 0
steer = 0
print(r.getObservationDimension())


7


In [ ]:
while(1):
    keys = p.getKeyboardEvents()
    for k,v in keys.items():
        if (k == p.B3G_UP_ARROW and (v&p.KEY_WAS_TRIGGERED)):
            speed = speed + 1 
            r.applyAction([speed,steer])
        if (k == p.B3G_DOWN_ARROW and (v&p.KEY_WAS_TRIGGERED)):
            speed = speed - 1 
            r.applyAction([speed,steer])
        if (k == p.B3G_LEFT_ARROW and (v&p.KEY_WAS_TRIGGERED)):
            steer = steer - 1 
            r.applyAction([speed,steer])
        if (k == p.B3G_RIGHT_ARROW and (v&p.KEY_WAS_TRIGGERED)):
            steer = steer + 1 
            r.applyAction([speed,steer])